In [444]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

##Read in the core parameters taken from Overseer

In [445]:
#ReadData
CoreParams = pd.read_excel('.\CropCoeff.xlsx', sheetname='CoreParams')
#Fix names
CoreParams['Crop'] = [str(CoreParams.ix[X,'Crop']).replace(" (","_") for X in range(CoreParams.index.size)]
CoreParams['Crop'] = [str(CoreParams.ix[X,'Crop']).replace("(","_") for X in range(CoreParams.index.size)]
CoreParams['Crop'] = [str(CoreParams.ix[X,'Crop']).replace(")","") for X in range(CoreParams.index.size)]
CoreParams['Crop'] = [str(CoreParams.ix[X,'Crop']).replace("/","_") for X in range(CoreParams.index.size)]
CoreParams['Crop'] = [str(CoreParams.ix[X,'Crop']).replace(" ","") for X in range(CoreParams.index.size)]
#SetIndex
CoreParams.set_index('Crop', inplace=True,drop=True)
#some useful lists
SCUMParamList = ['[Root].MaximumRootDepth.Value',
                '[Root].KLModifier.XYPairs.Y' ,
                '[Stover].CoverFunction.Expanding.SigCoverFunction.Xo.Value' ,
                '[Stover].CoverFunction.Expanding.SigCoverFunction.b.Value',
                '[Stover].CoverFunction.Expanding.SigCoverFunction.Ymax.Value' ,
                '[Stover].XoBiomass.Value',
                '[Stover].bBiomass.Value' ,
                '[Product].ExpectedYield.Value',
                '[Root].RootProportion.Value' ,
                '[Product].HarvestIndex.Value' ,
                '[Product].DryMatterContent.Value' ,
                '[Phenology].CanopyExpanding.Target.Value' ,
                '[Phenology].YieldIncreasing.Target.Value',
                '[Phenology].Senescing.Target.Value',
                '[Stover].NConc.Value',
                '[Product].MinimumNConc.Value',
                '[Product].MaximumNConc.Value',
                '[Root].MinimumNConc.Value',
                '[Root].MaximumNConc.Value',
                '[Nodule].NFixationOption']
CropList = CoreParams.index.tolist()
CropList.sort()
SCUMParamList.sort()

##Set up working dataframes

In [446]:
Step1 = CoreParams.ix[:,~(CoreParams.columns == 'TT_Sen')].copy()
SCUMParams = Step1.ix[:,~(Step1.columns == 'TT_Mat')].copy()
SCUMParams['[Nodule].NFixationOption'] = [str(SCUMParams.ix[X,'[Nodule].NFixationOption']).replace("'","") for X in range(SCUMParams.index.size)]
AncillaryParams = pd.DataFrame(index=CoreParams.index)

##Do some simple calculations first

In [447]:
#Calculate the inflection point ot the DM sigmoid as 50% of the crops duration
SCUMParams['[Stover].XoBiomass.Value'] = np.round(CoreParams.TT_Mat*0.5,0)
#The curvature parameter of the DM sigmoid is 1/5 of the inflection point
SCUMParams['[Stover].bBiomass.Value'] = np.round(SCUMParams['[Stover].XoBiomass.Value']*0.2,0)
#The duration of the Phase from the start to end of senescence is the total tt accumulated to mat less that to sen
SCUMParams['[Phenology].Senescing.Target.Value'] = CoreParams['TT_Mat'] - CoreParams['TT_Sen']                                                       

##Then calculate some additional values that will be needed for other parameters

In [448]:
#The total amount of biomass if the yield of the crop reached the expected value
AncillaryParams['YMax'] = (SCUMParams['[Product].ExpectedYield.Value']* \
                          SCUMParams['[Product].DryMatterContent.Value'])/  \
                          (1-SCUMParams['[Root].RootProportion.Value'])/ \
                          SCUMParams['[Product].HarvestIndex.Value']

#The amount of DM at maturity will be slightly less than YMax because the sigmoid won't have reached its asymtote by then
AncillaryParams['DMatMatur'] = AncillaryParams['YMax'] * \
                               1/(1+np.exp(-(CoreParams['TT_Mat']-\
                                             SCUMParams['[Stover].XoBiomass.Value'])/\
                                           SCUMParams['[Stover].bBiomass.Value']))
#Root DM at this point is a function of root Proportion
AncillaryParams['RootDM'] =  AncillaryParams['DMatMatur'] * SCUMParams['[Root].RootProportion.Value']
#Product DM at this point is a function of harvest index
AncillaryParams['ProductDM'] = (AncillaryParams['DMatMatur'] - \
                                AncillaryParams['RootDM']) * \
                                SCUMParams['[Product].HarvestIndex.Value']
#Stover DM is the residual
AncillaryParams['StoverDM'] = AncillaryParams['DMatMatur'] - \
                              AncillaryParams['RootDM'] - \
                              AncillaryParams['ProductDM']
#We are assuming 20% of the stover biomass is produced during the reproductive phase
AncillaryParams['StoverInRepro'] = AncillaryParams['StoverDM'] * 0.2
#So the amount of biomass at the start of the reproductive phase will be 80% of the total stover 
#adjusted to account for root proportion
AncillaryParams['DMatStartRepro'] = (AncillaryParams['StoverDM'] - \
                                    AncillaryParams['StoverInRepro']) * \
                                    (1+SCUMParams['[Root].RootProportion.Value'])
#Then we can invert the DM sigmoid to work out how much Tt has passed when this amout of biomass has accumulated
AncillaryParams['TtatStartRepro'] = np.log(1/(AncillaryParams['DMatStartRepro']/\
                                          AncillaryParams['DMatMatur'])-1) * \
                                        SCUMParams['[Stover].bBiomass.Value'] * -1 + \
                                        SCUMParams['[Stover].XoBiomass.Value']

##Calculate remaining Phenology parameters

In [449]:
#The tt duration of the canopy expanding (vegetative) phase
SCUMParams['[Phenology].CanopyExpanding.Target.Value'] = np.round(AncillaryParams['TtatStartRepro'],0)
#The tt duration of the yield increasing phase (growing reproductive DM and canopy not senescing)
#is what is left of thermal time to start of senescence after the vegetative phase is taken out
SCUMParams['[Phenology].YieldIncreasing.Target.Value'] = np.round(CoreParams['TT_Sen'] - SCUMParams['[Phenology].CanopyExpanding.Target.Value'],0)

##Generate arays of kl modifier values, adjusted for root depth

In [450]:
def StringMaker(DepthInts,RootDepth):
    YArray = ''
    for Depth in DepthInts:
        if Depth > 0:
            YArray += ','
        if Depth < RootDepth:
            klFact = 1*np.exp(-.002*(1500/RootDepth)*Depth)
        else:
            klFact = 0
        klFact = np.round(klFact,3)
        YArray += str(klFact)
    return YArray

DepthInts = range(0,1499,50)
SCUMParams['[Root].KLModifier.XYPairs.Y'] = \
    [StringMaker(DepthInts,SCUMParams.ix[X,'[Root].MaximumRootDepth.Value']) \
     for X in range(SCUMParams.index.size)]

##Build XML and write to file

In [451]:
Crops = ET.Element('Crops')
for Crop in CropList:
    CropElement = ET.SubElement(Crops, 'Cultivar')
    NameTag = ET.SubElement(CropElement, 'Name')
    NameTag.text = str(Crop)
    for Param in SCUMParamList:
        Command = ET.SubElement(CropElement,'Command')
        Command.text = str(Param + ' = ' + str(SCUMParams.ix[Crop,Param]))

ET.ElementTree(Crops).write('.\Crop.xml')